# Running a program remotely

In this tutorial, we will write a "Hello World" program using Quantum Serverless. The program will be a Qiskit example that prepares a Bell state and prints the measured probability distribution.

### Writing the Program

First, we need to write the program code and save it to a file called [program_1.py](./source_files/program_1.py). 
The code for the program is shown below:

```python
# source_files/program_1.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

sampler = Sampler()

quasi_dists = sampler.run(circuit).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")
```

This program creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and prints the measured probability distribution.

### Running the Program

To run the program, we need to import the necessary classes and configure them. One of these classes is QuantumServerless, which is a client class for interacting with compute resources.

QuantumServerless takes a Provider object as a constructor argument. The Provider object stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the program locally on our machine. If you want to run the program elsewhere, you will need to provide the corresponding host and authentication details.

In [3]:
from quantum_serverless import QuantumServerless, GatewayProvider
import os

In [5]:
provider = GatewayProvider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

After importing the necessary classes and configuring them, we can run the program by calling the `run()` method of the `QuantumServerless` object:

`Program` accepts couple of required parameters:
- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - folder where  your script is located. This is optional parameter and will be current folder by default. 

In [12]:
from quantum_serverless import Program

program = Program(
    title="First program",
    entrypoint="program_1.py",
    working_dir="./source_files/"
)

job = serverless.run(program)
job

<Job | 3bbb5ced-06d7-44ff-a08c-58266881d54c>

As result of `run` call we get `Job` which has `status` method to check status of program execution, `logs` to get logs of execution.

In [15]:
job.status()

'SUCCEEDED'

In [16]:
job.logs()

'Quasi distribution: {0: 0.4999999999999999, 3: 0.4999999999999999}\n'

Also this object has `job_id` property that can be used if you want to access job results later.
To do so we need to call `get_job_by_id` method of `QuantumServerless` client.

In [17]:
job.job_id

'3bbb5ced-06d7-44ff-a08c-58266881d54c'

Users can fetch previously ran jobs from configured providers.

In [18]:
serverless.get_job_by_id(job.job_id)

<Job | 3bbb5ced-06d7-44ff-a08c-58266881d54c>

To get a list of previously ran jobs client has `get_jobs` method.

In [21]:
serverless.get_jobs()

[<Job | 9ad485a3-9537-4098-8253-bdc7146cd8c9>,
 <Job | 89a97a04-bf1c-4389-b245-5e6d017b0dd7>,
 <Job | 722707ff-1bfa-4f51-8cc8-bef96be88f6a>,
 <Job | 5bee2875-24e5-42c8-964f-9b55e300fd54>,
 <Job | de5e8dd3-e622-4e3e-9eaf-dd96c93299dd>,
 <Job | 3bbb5ced-06d7-44ff-a08c-58266881d54c>]